In [3]:
# Let's first load and combine the provided notebooks and Python script into a structured and streamlined code.
# We'll begin by integrating the user overview, engagement, and experience analysis into one cohesive Python script.

# Loading necessary libraries for the tasks
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sqlalchemy import create_engine
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Task 1 - User Overview Analysis

# Data loading function from telecom_analysis.py script
def load_data_using_sqlalchemy(query):
    """
    Connects to the PostgreSQL database and loads data based on the provided SQL query using SQLAlchemy.
    """
    try:
        # Connection string fetched from environment variables
        connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

        # SQLAlchemy engine creation
        engine = create_engine(connection_string)

        # Load data into a pandas DataFrame
        df = pd.read_sql_query(query, engine)

        # Handle missing values by filling them with 0
        df.fillna(0, inplace=True)

        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Load environment variables (for connecting to PostgreSQL)
load_dotenv()

# PostgreSQL database parameters
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Load data from PostgreSQL database
query = "SELECT * FROM xdr_data;"  # Example query, you can adjust it as needed
df = load_data_using_sqlalchemy(query)

# Check if the data was loaded successfully
if df is not None:
    print("Data loaded successfully.")
else:
    print("Failed to load data.")


# Task 1.1 - Aggregating User Behavior Data
def clean_and_aggregate_user_behavior(df):
    """
    Clean the data by handling missing values and ensuring the necessary columns are numeric.
    Then aggregate per user the number of sessions, session duration, download/upload data, and data volume for each application.
    """
    # Convert necessary columns to numeric, forcing errors to NaN
    df['Total DL (Bytes)'] = pd.to_numeric(df['Total DL (Bytes)'], errors='coerce')
    df['Total UL (Bytes)'] = pd.to_numeric(df['Total UL (Bytes)'], errors='coerce')
    df['Dur. (ms)'] = pd.to_numeric(df['Dur. (ms)'], errors='coerce')

    # Handle missing values by filling NaN with 0 for numeric columns
    df['Total DL (Bytes)'].fillna(0, inplace=True)
    df['Total UL (Bytes)'].fillna(0, inplace=True)
    df['Dur. (ms)'].fillna(0, inplace=True)

    # Ensure there are no negative values (if necessary)
    df['Total DL (Bytes)'] = df['Total DL (Bytes)'].abs()
    df['Total UL (Bytes)'] = df['Total UL (Bytes)'].abs()

    # Perform aggregation
    df_user_behavior = df.groupby("MSISDN/Number").agg(
        session_count=('MSISDN/Number', 'count'),
        total_session_duration=('Dur. (ms)', 'sum'),
        total_download=('Total DL (Bytes)', 'sum'),
        total_upload=('Total UL (Bytes)', 'sum'),
        total_data_volume=(('Total DL (Bytes)', 'sum'), ('Total UL (Bytes)', 'sum'))
    ).reset_index()

    # Adding total data volume (download + upload)
    df_user_behavior['total_data_volume'] = df_user_behavior['total_download'] + df_user_behavior['total_upload']

    return df_user_behavior

# Perform data cleaning and aggregation
df_user_behavior = clean_and_aggregate_user_behavior(df)


# Task 1.2 - Exploratory Data Analysis (Summary)
def exploratory_data_analysis(df_user_behavior):
    """
    Conduct exploratory data analysis on the aggregated user data and return summary statistics.
    """
    print("Summary Statistics:")
    print(df_user_behavior.describe())

    # Plot histograms for session duration, download, upload, and data volume
    df_user_behavior[['total_session_duration', 'total_download', 'total_upload', 'total_data_volume']].hist(figsize=(12, 10))
    plt.show()

exploratory_data_analysis(df_user_behavior)

# Task 2 - User Engagement Analysis
# Aggregate per customer engagement metrics and normalize for k-means
def user_engagement_analysis(df):
    # Aggregating engagement metrics
    engagement_metrics = df.groupby('MSISDN/Number').agg(
        session_frequency=('MSISDN/Number', 'count'),
        total_duration=('Dur. (ms)', 'sum'),
        total_traffic=('Total DL (Bytes)', 'sum') + df['Total UL (Bytes)']
    ).reset_index()

    # Normalize metrics
    scaler = StandardScaler()
    engagement_metrics[['session_frequency', 'total_duration', 'total_traffic']] = scaler.fit_transform(
        engagement_metrics[['session_frequency', 'total_duration', 'total_traffic']])

    # K-means clustering (k=3)
    kmeans = KMeans(n_clusters=3, random_state=0)
    engagement_metrics['engagement_cluster'] = kmeans.fit_predict(
        engagement_metrics[['session_frequency', 'total_duration', 'total_traffic']])

    return engagement_metrics

# Perform User Engagement Analysis
df_engagement = user_engagement_analysis(df)
print(df_engagement.head())

# Task 3 - Experience Analytics
def experience_analytics(df):
    """
    Conduct user experience analysis based on network performance metrics and device characteristics.
    """
    # Aggregate network performance data
    df_experience = df.groupby('MSISDN/Number').agg(
        avg_tcp_retransmission=('TCP DL Retrans. Vol (Bytes)', 'mean'),
        avg_rtt=('Avg RTT DL (ms)', 'mean'),
        avg_throughput=('Avg Bearer TP DL (kbps)', 'mean'),
        handset_type=('Handset Type', 'first')
    ).reset_index()

    # Handle missing values (fill with mean or mode)
    df_experience.fillna(df_experience.mean(numeric_only=True), inplace=True)

    return df_experience

# Perform Experience Analytics
df_experience = experience_analytics(df)
print(df_experience.head())

# Task 3.4 - K-Means Clustering for Experience Analysis (k=3)
def experience_clustering(df_experience):
    features = df_experience[['avg_throughput', 'avg_tcp_retransmission', 'avg_rtt']].copy()
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    kmeans = KMeans(n_clusters=3, random_state=0)
    df_experience['experience_cluster'] = kmeans.fit_predict(scaled_features)
    
    return df_experience, kmeans.cluster_centers_

df_experience_clustered, experience_cluster_centers = experience_clustering(df_experience)

print("Experience Cluster Centers:")
print(pd.DataFrame(experience_cluster_centers, columns=['Avg Throughput', 'Avg TCP Retransmission', 'Avg RTT']))

# Combining Task 1, 2, and 3 for an overall report
df_combined = df_user_behavior.merge(df_engagement, on='MSISDN/Number').merge(df_experience_clustered, on='MSISDN/Number')

print("Combined Data Head:")
print(df_combined.head())




Data loaded successfully.


C:\Users\Yonas\AppData\Local\Temp\ipykernel_17900\3111102901.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Total DL (Bytes)'].fillna(0, inplace=True)
C:\Users\Yonas\AppData\Local\Temp\ipykernel_17900\3111102901.py:81: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

KeyError: "Column(s) [('Total DL (Bytes)', 'sum')] do not exist"

In [ ]:
from sklearn.metrics import euclidean_distances
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Task 4.1 - Assign Engagement and Experience Scores
def calculate_satisfaction_scores(df, engagement_cluster_center, experience_cluster_center):
    # Calculate the Euclidean distance for engagement score (less engaged cluster)
    df['engagement_score'] = euclidean_distances(
        df[['session_frequency', 'total_duration', 'total_traffic']], [engagement_cluster_center]
    ).flatten()

    # Calculate the Euclidean distance for experience score (worst experience cluster)
    df['experience_score'] = euclidean_distances(
        df[['avg_throughput', 'avg_tcp_retransmission', 'avg_rtt']], [experience_cluster_center]
    ).flatten()

    return df

# Use the cluster centers from previous tasks
engagement_cluster_center = df_engagement[['session_frequency', 'total_duration', 'total_traffic']].mean(axis=0).values
experience_cluster_center = experience_cluster_centers[2]  # Assuming the 3rd cluster is the worst experience

# Assign the scores
df_combined = calculate_satisfaction_scores(df_combined, engagement_cluster_center, experience_cluster_center)

# Task 4.2 - Calculate satisfaction score as the average of engagement and experience scores
df_combined['satisfaction_score'] = (df_combined['engagement_score'] + df_combined['experience_score']) / 2

# Report top 10 satisfied customers
top_10_satisfied_customers = df_combined.nlargest(10, 'satisfaction_score')
print("Top 10 Satisfied Customers:")
print(top_10_satisfied_customers[['MSISDN/Number', 'satisfaction_score']])

# Task 4.3 - Build a regression model to predict satisfaction score
X = df_combined[['engagement_score', 'experience_score']]
y = df_combined['satisfaction_score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predict satisfaction score
y_pred = regressor.predict(X_test)

# Evaluate the regression model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error of Regression: {mse}")

# Task 4.4 - Run k-means clustering (k=2) on engagement and experience scores
kmeans = KMeans(n_clusters=2, random_state=0)
df_combined['kmeans_cluster'] = kmeans.fit_predict(df_combined[['engagement_score', 'experience_score']])

# Task 4.5 - Aggregate the average satisfaction & experience score per cluster
cluster_summary = df_combined.groupby('kmeans_cluster').agg({
    'satisfaction_score': 'mean',
    'experience_score': 'mean',
    'engagement_score': 'mean'
}).reset_index()

print("Cluster Summary (Average Satisfaction, Engagement, and Experience Scores):")
print(cluster_summary)
